In [64]:
from surprise import Dataset
import pandas as pd
from surprise import Dataset, Reader
from surprise import Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import gzip



In [65]:
opinion = pd.read_csv(r'data_set_limpio/reviews_preparado.csv.gz')

In [66]:
df = pd.read_csv(r'data_set_limpio/games_preparado.csv.gz')

In [67]:
prueba1 = pd.merge(opinion[['item_id','user','recommend','opinion']], df[['id','app_name']], left_on='item_id', right_on='id')

prueba1['recommend'] = prueba1['recommend'].replace({True: 1, False: 0})

In [68]:
for index, row in prueba1.iterrows():
    if (row[2] == 1) and (row[3] == 2):
        prueba1.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 1):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 1) and (row[3] == 0):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 2):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 0):
        prueba1.at[index, 'rating'] = 0

C:\Users\jonat\AppData\Local\Temp\ipykernel_7532\2034525896.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[2] == 1) and (row[3] == 2):
C:\Users\jonat\AppData\Local\Temp\ipykernel_7532\2034525896.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[2] == 1) and (row[3] == 2):
C:\Users\jonat\AppData\Local\Temp\ipykernel_7532\2034525896.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (row[2] == 1) and (row[

In [69]:
prueba1

,item_id,user,recommend,opinion,id,app_name,rating
0,1250,76561197970982479,1,2,1250,Killing Floor,3.0
1,1250,death-hunter,1,2,1250,Killing Floor,3.0
2,1250,DJKamBer,1,1,1250,Killing Floor,2.0
3,1250,diego9031,1,0,1250,Killing Floor,1.0
4,1250,76561198081962345,1,0,1250,Killing Floor,1.0
...,...,...,...,...,...,...,...
49590,307130,llDracuwulf,1,2,307130,Asteria,3.0
49591,209120,ChrisCoroner,1,2,209120,Street Fighter X Tekken,3.0
49592,220090,MeloncraftLP,1,0,220090,The Journey Down: Chapter One,1.0
49593,262850,MeloncraftLP,1,0,262850,The Journey Down: Chapter Two,1.0


In [70]:
entrenar = prueba1[['user','app_name','rating']]

In [101]:
# Definir un lector para el formato de tus datos
reader = Reader(rating_scale=(0, 3))  # Adaptar rating_scale a tus datos

# Cargar los datos con el lector
data = Dataset.load_from_df(entrenar, reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.1)  # Puedes ajustar el tamaño de prueba

# Seleccionar un algoritmo (por ejemplo, SVD) y entrenar el modelo
model = SVD()
model.fit(trainset)

# Realizar predicciones en el conjunto de prueba
predictions = model.test(testset)

# Evaluar las predicciones
accuracy.rmse(predictions)

RMSE: 0.9163


0.9163361010750946

In [93]:
from surprise import KNNBasic

# Definir un lector para el formato de tus datos
reader = Reader(rating_scale=(0, 3))  # Adaptar rating_scale a tus datos

# Cargar los datos con el lector
data = Dataset.load_from_df(entrenar, reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.3)  # Puedes ajustar el tamaño de prueba

# Seleccionar un algoritmo (por ejemplo, KNN) y entrenar el modelo
model = KNNBasic()  # Seleccionar el algoritmo KNN
model.fit(trainset)

# Realizar predicciones en el conjunto de prueba
predictions = model.test(testset)

# Evaluar las predicciones
accuracy.rmse(predictions)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9882


0.9882134417709891

In [102]:
# ID del usuario para el cual quieres obtener recomendaciones
usuario_especifico = '76561197970982479'  # Reemplaza 'js41637' con el ID del usuario para el que quieres las recomendaciones

# Crear una lista de juegos ya valorados por el usuario específico
juegos_valorados = prueba1[prueba1['user'] == usuario_especifico]['app_name'].unique()

# Crear una lista de todos los juegos disponibles
todos_los_juegos = prueba1['app_name'].unique()

# Crear una lista de juegos no valorados por el usuario específico
juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

# Generar predicciones para los juegos no valorados por el usuario
predicciones = [model.predict(usuario_especifico, juego) for juego in juegos_no_valorados]

# Ordenar las predicciones en base a la valoración y obtener los juegos recomendados
recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:10]  # Obtener las 5 mejores recomendaciones

# Mostrar los juegos recomendados
for recomendacion in recomendaciones:
    print(f"Juego: {recomendacion.iid}, Valoración estimada: {recomendacion.est}")


Juego: Mass Effect 2, Valoración estimada: 2.9278932721441837
Juego: King Arthur's Gold, Valoración estimada: 2.916207784708064
Juego: Dust: An Elysian Tail, Valoración estimada: 2.9006307541850513
Juego: Gunpoint, Valoración estimada: 2.8886750355282027
Juego: Psychonauts, Valoración estimada: 2.8880674976435556
Juego: RimWorld, Valoración estimada: 2.8828020742537612
Juego: CrossCode, Valoración estimada: 2.8545041203214945
Juego: Alien Swarm, Valoración estimada: 2.853988808878739
Juego: Burnout Paradise: The Ultimate Box, Valoración estimada: 2.8459847618091114
Juego: SNOW, Valoración estimada: 2.843880454332686


In [104]:
prueba1.loc[prueba1['user']== '76561197970982479']

,item_id,user,recommend,opinion,id,app_name,rating
0,1250,76561197970982479,1,2,1250,Killing Floor,3.0
334,22200,76561197970982479,1,2,22200,Zeno Clash,3.0


In [57]:
prueba1.to_csv('data_set_limpio/modelo.csv',encoding='UTF-8',index=False)

In [63]:
with open('data_set_limpio/modelo.csv', 'rb') as f_in, gzip.open('data_set_limpio/modelo.csv.gz', 'wb') as f_out:
    f_out.writelines(f_in)